<a href="https://colab.research.google.com/github/YoheiFukuhara/recommender-system/blob/main/11_preprocess2sparse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle

import pandas as pd
from google.colab import drive
from scipy.sparse import lil_matrix, csr_matrix, save_npz, load_npz
from sklearn.model_selection import train_test_split

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
BASE_PATH = '/content/drive/MyDrive/ColabNotebooks/ML/Recommend/output/'
df = pd.read_csv(BASE_PATH+'very_small_rating.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1982718 entries, 0 to 1982717
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   movie_idx  int64  
dtypes: float64(1), int64(3)
memory usage: 60.5 MB


In [3]:
N = df.userId.max() + 1 # number of users
M = df.movie_idx.max() + 1 # number of movies

In [4]:
df_train, df_test = train_test_split(df, train_size=0.8, random_state=42)

In [5]:
A = lil_matrix((N, M))
print("Calling: update_train")
count = 0

Calling: update_train


In [6]:
%%time
def update_train(row):
    global count
    count += 1
    if count % 100000 == 0:
        print("processed: %.3f" % (float(count)/len(df)*0.8)) # 多分間違っている

    i = int(row.userId)
    j = int(row.movie_idx)
    A[i,j] = row.rating
df_train.apply(update_train, axis=1)

processed: 0.040
processed: 0.081
processed: 0.121
processed: 0.161
processed: 0.202
processed: 0.242
processed: 0.282
processed: 0.323
processed: 0.363
processed: 0.403
processed: 0.444
processed: 0.484
processed: 0.525
processed: 0.565
processed: 0.605
CPU times: user 56.2 s, sys: 922 ms, total: 57.1 s
Wall time: 57.2 s


In [7]:
save_npz(BASE_PATH+"Atrain.npz", A.tocsr())

In [8]:
# test ratings dictionary
A_test = lil_matrix((N, M))
print("Calling: update_test")
count = 0

Calling: update_test


In [9]:
%%time
def update_test(row):
    global count
    count += 1
    if count % 100000 == 0:
        print("processed: %.3f" % (float(count)/len(df_test)))

    i = int(row.userId)
    j = int(row.movie_idx)
    A_test[i,j] = row.rating
df_test.apply(update_test, axis=1)

processed: 0.252
processed: 0.504
processed: 0.757
CPU times: user 14 s, sys: 84.9 ms, total: 14.1 s
Wall time: 14.1 s


In [10]:
save_npz(BASE_PATH+"Atest.npz", A_test.tocsr())